In [12]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [13]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [ ]:
from tqdm import tqdm
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [37]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [23]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/',
    search_pattern_re='stat.*npy',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230531/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230531/scanimage_data/zstack/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230523/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230523/scanimage_data/zstack/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230517/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230517/scanimage_data/zstack/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230515/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_03

In [24]:
paths_found = [p for p in paths_found if 'exp' in p]

In [26]:
dates_toUse = natsort.natsorted([Path(p).parts[-7] for p in paths_found])
dates_toUse

['20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230510',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230528',
 '20230529',
 '20230530',
 '20230531',
 '20230601']

In [33]:
[sftp.get(str(Path(p).parent / 'stat.npy'), str(Path(r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/stat_and_ops') / Path(p).parts[-7] / 'stat.npy')) for p in paths_found[2:]]

100%|█████████████████████████████████████████████████████████████▉| 25.8M/25.8M [00:02<00:00, 11.9Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 24.4M/24.5M [00:02<00:00, 10.3Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 26.8M/26.8M [00:03<00:00, 7.40Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 27.0M/27.0M [00:03<00:00, 7.46Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 26.3M/26.3M [00:03<00:00, 8.74Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 25.3M/25.3M [00:02<00:00, 11.0Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 26.9M/26.9M [00:02<00:00, 10.3Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 26.7M/26.7M [00:03<00:00, 8.69Mb/s]
100%|█████████████████████████████████████████████████████████████▉| 25.5M/25.5M [00:03<00:00, 7.09Mb/s]
100%|██████████████████████████████████████████████████

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()